# $\bm{16}$ $\enspace$ **PyTorch 神经网络基础**

## $\bm{16.1}$ $\enspace$ **层和块**

在构造自定义块之前，我们先回顾一下多层感知机的代码。下面的代码生成一个网络，其中包含一个具有 $256$ 个单元和 $\mathrm{ReLU}$ 激活函数的全连接隐藏层，然后是一个具有 $10$ 个隐藏单元且不带激活函数的全连接输出层。

In [4]:
import torch
from torch import nn
from torch.nn import functional as F


net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.2445, -0.2127,  0.1140, -0.1277, -0.0654,  0.2452, -0.0732, -0.0911,
         -0.1354,  0.0579],
        [ 0.1722, -0.1546,  0.0979, -0.0508, -0.1370,  0.1456,  0.0481, -0.0766,
         -0.1437,  0.0245]], grad_fn=<AddmmBackward0>)

在这个例子中，我们通过实例化 `nn.Sequential` 来构建我们的模型，层的执行顺序是作为参数传递的。`nn.Sequential` 定义了一种特殊的 `Module`，即在 PyTorch 中表示了一个块的类，它维护了一个由 `Module` 组成的有序列表。注意，两个全连接层都是都是 `Linear` 类的实例，`Linear` 类本身就是 `Module` 的子类。另外，到目前为止，我们一直在通过 `net(X)` 调用我们的模型来获得输出。这实际上是 `net.__call__(X)` 的简写。这个前向传播函数非常简单：它将列表中的每个块连接在一起，将每个块的输出作为下个块的输入。

### $\bm{16.1.1}$ $\enspace$ **自定义块**

实现自定义块，我们需要总结一下每个块必须提供的基本功能：

$1$. $\enspace$ 将输入数据作为其前向传播函数的参数。

$2$. $\enspace$ 通过前向传播函数来生成输出，需要注意的是输出的形状可能与输入的形状不同。

$3$. $\enspace$ 计算其输出关于输入的梯度，可以通过其反向传播函数进行访问。

$4$. $\enspace$ 存储和访问前向传播计算所需的参数。

$5$. $\enspace$ 根据需要初始化模型参数。

在下面的代码片段中，我们从零开始编写一个块。它包含一个多层感知机，其具有 $256$ 个隐藏单元的隐藏层和一个 $10$ 为输出层。注意，下面的 `MLP` 类继承了表示块的类。我们的实现只需要提供我们自己的构造函数（Python 中的 `__init__` 函数）和前向传播函数。

In [5]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来进行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入x返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义
        return self.out(F.relu(self.hidden(X)))

我们首先看一下前向传播函数，它以 `X` 作为输入，计算带有激活函数的隐藏表示，并输出其未规范化的输出值。在这个 MLP 实现中，两个层都是实例变量。

接着我们实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层。注意一些关键细节：首先，我们定制的 `__init__` 函数通过 `super().__init()__()` 调用父类的 `__init__` 函数；然后，我们实例化两个全连接层，分别为 `self.hidden` 和 `self.out`。

In [6]:
net = MLP()
net(X)

tensor([[ 0.0218,  0.2706, -0.3561,  0.0540,  0.2832, -0.0396, -0.1182,  0.0032,
          0.0818,  0.1952],
        [-0.0769,  0.1141, -0.4075,  0.0018,  0.3676, -0.0809, -0.1673,  0.2259,
          0.0233,  0.3117]], grad_fn=<AddmmBackward0>)